<h3>Encoder-Decoder with Attention</h3>

The encoder-decoder model for recurrent neural networks is an architecture for sequence-to-sequence prediction problems.

It is comprised of two sub-models, as its name suggests:

    Encoder: The encoder is responsible for stepping through the input time steps and encoding the entire sequence into a fixed length vector called a context vector.
    Decoder: The decoder is responsible for stepping through the output time steps while reading from the context vector.

A problem with the architecture is that performance is poor on long input or output sequences. The reason is believed to be because of the fixed-sized internal representation used by the encoder.

Attention is an extension to the architecture that addresses this limitation. It works by first providing a richer context from the encoder to the decoder and a learning mechanism where the decoder can learn where to pay attention in the richer encoding when predicting each time step in the output sequence.

In [30]:
from random import randint
from numpy import array
from numpy import argmax
from keras.models import Sequential
from keras.layers import RepeatVector, LSTM, TimeDistributed, Dense
from attention_decoder import AttentionDecoder

In [5]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(0, n_unique-1) for _ in range(length)]
 
# one hot encode sequence
def one_hot_encode(sequence, n_unique):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return array(encoding)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

# prepare data for the LSTM
def get_pair(n_in, n_out, n_unique):
    # generate random sequence
    sequence_in = generate_sequence(n_in, n_unique)
    sequence_out = sequence_in[:n_out] + [0 for _ in range(n_in-n_out)]
    # one hot encode
    X = one_hot_encode(sequence_in, n_unique)
    y = one_hot_encode(sequence_out, n_unique)
    # reshape as 3D
    X = X.reshape((1, X.shape[0], X.shape[1]))
    y = y.reshape((1, y.shape[0], y.shape[1]))
    return X,y
 
# generate random sequence
X, y = get_pair(5, 2, 50)
print(X.shape, y.shape)
print('X=%s, y=%s' % (one_hot_decode(X[0]), one_hot_decode(y[0])))

(1, 5, 50) (1, 5, 50)
X=[37, 16, 10, 14, 43], y=[37, 16, 0, 0, 0]


In [9]:
X.shape, y.shape

((1, 5, 50), (1, 5, 50))

In [11]:
X

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0, 0, 0]]])

In [12]:
y

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]])

In [13]:
# configure problem
n_features = 50
n_timesteps_in = 5
n_timesteps_out = 2

<b>RepeatVector</b>: This layer takes in the input from the previous layer and keeps repeating it again and again. Hence it provides with the same encoding of the sentence to the Dense Layer. That finally outputs the results.
<br>
<b>Time Distributed</b>: This is explained by the means of another notebook

In [18]:
# define model
model = Sequential()
model.add(LSTM(150, input_shape=(n_timesteps_in, n_features)))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(150, return_sequences=True))
model.add(TimeDistributed(Dense(n_features, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

This is how you can train a model using a for loop with one example each

In [19]:
# train LSTM
for epoch in range(5000):
    # generate new random sequence
    X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    # fit model for one epoch on this sequence
    model.fit(X, y, epochs=1, verbose=2)

Epoch 1/1
 - 1s - loss: 3.9149 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8993 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8843 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8879 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8730 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8666 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8571 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8323 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8356 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8093 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7904 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7211 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7325 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6968 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6177 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5024 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.4448 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.4487 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.2171 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 3.2400 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.1617 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 2.8035 - acc: 0.6000
Epoch 

Epoch 1/1
 - 0s - loss: 1.6891 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6373 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6268 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6504 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7222 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6269 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4019 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5951 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6037 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6739 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5426 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6373 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6401 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6945 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6107 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5208 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6397 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5444 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5502 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6114 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6059 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6254 - acc: 0.6000
Epoch 1/1


 - 0s - loss: 1.6206 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6593 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5773 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6111 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5721 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5974 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5264 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4580 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5649 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6686 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5436 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5366 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5605 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6014 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6051 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5650 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4654 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5929 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5842 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5321 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6268 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6212 - acc: 0.6000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 1.5388 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5932 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6550 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6829 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7062 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7199 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5501 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5726 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5135 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5528 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4733 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7031 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6700 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5653 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5311 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5840 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5000 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5167 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5984 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7132 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5161 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5603 - acc: 0.6000
Epoch 1/1


 - 0s - loss: 1.2247 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5366 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5099 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6331 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4566 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6871 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5625 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5446 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5025 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5180 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6168 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6130 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5754 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6109 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5257 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6252 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5953 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5374 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5103 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6267 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5977 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6377 - acc: 0.6000
Epoch 1/1
 - 0s - lo

...

Epoch 1/1
 - 0s - loss: 1.2553 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.6762 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6140 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7254 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7306 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9361 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3225 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5259 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6529 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6445 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7180 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1342 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4629 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4890 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6338 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7954 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8668 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7313 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.4229 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8123 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5382 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3995 - acc: 1.0000
Epoch 1/1


 - 0s - loss: 0.3929 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4595 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1968 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1847 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0422 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5631 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6008 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4107 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7249 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5641 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7853 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5789 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9540 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1460 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4796 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8050 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4969 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7219 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3341 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6510 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0908 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3610 - acc: 0.8000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.9051 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1867 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4251 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8296 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6610 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5747 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8387 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5355 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3851 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2180 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3362 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4335 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3865 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1757 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.6926 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2976 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5536 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3017 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7134 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7980 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2760 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7509 - acc: 0.8000
Epoch 1/1


 - 0s - loss: 0.2018 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4421 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4693 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7559 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6336 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7115 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5305 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4556 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8457 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5015 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2833 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4334 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5327 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4117 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2813 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7504 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3960 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5367 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2040 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1427 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7057 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0516 - acc: 0.8000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.4389 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3829 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5593 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8688 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1601 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.5092 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5835 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4526 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4727 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.4809 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2846 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.2159 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.9354 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3399 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5922 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1572 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7213 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.3968 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5917 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5421 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1392 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.7701 - acc: 0.8000
Epoch 1/1


In [21]:
for _ in range(10):
    X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    yhat = model.predict(X, verbose=0)
    print('Expected:', one_hot_decode(y[0]), 'Predicted', one_hot_decode(yhat[0]))

Expected: [4, 49, 0, 0, 0] Predicted [4, 21, 0, 0, 0]
Expected: [42, 12, 0, 0, 0] Predicted [42, 42, 0, 0, 0]
Expected: [42, 15, 0, 0, 0] Predicted [42, 33, 0, 0, 0]
Expected: [7, 21, 0, 0, 0] Predicted [7, 11, 0, 0, 0]
Expected: [15, 16, 0, 0, 0] Predicted [35, 35, 0, 0, 0]
Expected: [35, 8, 0, 0, 0] Predicted [35, 35, 0, 0, 0]
Expected: [46, 41, 0, 0, 0] Predicted [46, 43, 0, 0, 0]
Expected: [2, 41, 0, 0, 0] Predicted [2, 2, 0, 0, 0]
Expected: [37, 41, 0, 0, 0] Predicted [37, 33, 0, 0, 0]
Expected: [34, 18, 0, 0, 0] Predicted [34, 18, 0, 0, 0]


Now Let's See how good the model works with Attention:

In [31]:
# define model
model = Sequential()
model.add(LSTM(150, input_shape=(n_timesteps_in, n_features), return_sequences=True))
model.add(AttentionDecoder(150, n_features))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [32]:
# train LSTM
for epoch in range(5000):
    # generate new random sequence
    X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    # fit model for one epoch on this sequence
    model.fit(X, y, epochs=1, verbose=2)

Epoch 1/1
 - 2s - loss: 3.9051 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8957 - acc: 0.0000e+00
Epoch 1/1
 - 0s - loss: 3.8759 - acc: 0.4000
Epoch 1/1
 - 0s - loss: 3.8587 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8180 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.8256 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7689 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7500 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.7132 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6742 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.6511 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 3.6207 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.5726 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 3.6129 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.4680 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.4886 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.4215 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.3995 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.2376 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 3.1421 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 2.9604 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 2.8324 - acc: 0.6000
Ep

Epoch 1/1
 - 0s - loss: 1.6088 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7022 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6316 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5819 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6566 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6273 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5571 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5514 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.6493 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5217 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6592 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6144 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5310 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6865 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5608 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5129 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4831 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.7113 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4476 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4950 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.6589 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6651 - acc: 0.6000
Epoch 1/1


 - 0s - loss: 1.5191 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5193 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3188 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5952 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5197 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4969 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5249 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6347 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6107 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4282 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5092 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4376 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5169 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.6023 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5750 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5752 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3877 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.2315 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4944 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.5145 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2545 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.6365 - acc: 0.6000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 1.6616 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4909 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3447 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5362 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4516 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.2547 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3590 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2607 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.4263 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3725 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3703 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.4959 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2906 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.1174 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.3891 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.0939 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.1549 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2348 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.2457 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0484 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.6189 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.3977 - acc: 0.8000
Epoch 1/1


 - 0s - loss: 0.5886 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9133 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9354 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0014 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9016 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8598 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9188 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5317 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.8551 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6436 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 1.0334 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.1035 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6728 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.0584 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9423 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.9312 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.5388 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.7334 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.6652 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 1.5596 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 1.0700 - acc: 0.6000
Epoch 1/1
 - 0s - loss: 0.8443 - acc: 1.0000
Epoch 1/1
 - 0s - lo

...

 - 0s - loss: 0.0199 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0110 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0289 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2430 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0363 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0066 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1841 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0129 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0084 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0756 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0093 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0876 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0157 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0872 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0119 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0057 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0198 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2653 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0668 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0077 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4980 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.1274 - acc: 1.0000
Epoch 1/1
 - 0s - lo

 - 0s - loss: 0.0077 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0194 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0167 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0343 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0164 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0915 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2018 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0086 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0233 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0152 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0163 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0092 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0548 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0071 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0048 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0463 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0336 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1844 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0061 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0030 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0113 - acc: 1.0000
Epoch 1/1
 - 0s - lo

Epoch 1/1
 - 0s - loss: 0.2074 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0419 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0124 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0392 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.7457 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0088 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0948 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0155 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0103 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0027 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0097 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0876 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0223 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4920 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0063 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0062 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0030 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.4136 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0078 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0102 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0144 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0110 - acc: 1.0000
Epoch 1/1


Epoch 1/1
 - 0s - loss: 0.0132 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.3042 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0209 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0261 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0119 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0113 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0200 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0494 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0072 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0113 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0070 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0052 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0387 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0294 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1040 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1402 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1592 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0724 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0032 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0348 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0079 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0140 - acc: 1.0000
Epoch 1/1


 - 0s - loss: 0.3575 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0024 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 9.6249e-04 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0092 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.1015 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0777 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0040 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2148 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0039 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0205 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0029 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0184 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.6781 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0143 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0643 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.2302 - acc: 0.8000
Epoch 1/1
 - 0s - loss: 0.0395 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0183 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0123 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0136 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0079 - acc: 1.0000
Epoch 1/1
 - 0s - loss: 0.0493 - acc: 1.0000
Epoch 1/1
 - 0s 

In [33]:
for _ in range(10):
    X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
    yhat = model.predict(X, verbose=0)
    print('Expected:', one_hot_decode(y[0]), 'Predicted', one_hot_decode(yhat[0]))

Expected: [45, 41, 0, 0, 0] Predicted [45, 41, 0, 0, 0]
Expected: [34, 18, 0, 0, 0] Predicted [34, 18, 0, 0, 0]
Expected: [29, 4, 0, 0, 0] Predicted [29, 22, 0, 0, 0]
Expected: [31, 12, 0, 0, 0] Predicted [31, 12, 0, 0, 0]
Expected: [15, 9, 0, 0, 0] Predicted [15, 9, 0, 0, 0]
Expected: [47, 31, 0, 0, 0] Predicted [47, 31, 0, 0, 0]
Expected: [36, 15, 0, 0, 0] Predicted [36, 15, 0, 0, 0]
Expected: [0, 39, 0, 0, 0] Predicted [0, 5, 0, 0, 0]
Expected: [20, 2, 0, 0, 0] Predicted [20, 2, 0, 0, 0]
Expected: [37, 38, 0, 0, 0] Predicted [37, 38, 0, 0, 0]


As seen above the accuraccy in case of attention is higher as compared to encoder decoder model